In [1]:
#import os
#os.chdir("Applied Data Science\project2\code")
#os.getcwd()

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('data/yellow_tripdata_2019-02.csv')
df.dropna(inplace=True)
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-02-01 00:59:04,2019-02-01 01:07:27,1,2.1,1,N,48,234,1,9.0,0.5,0.5,2.0,0.0,0.3,12.3,0.0
1,1,2019-02-01 00:33:09,2019-02-01 01:03:58,1,9.8,1,N,230,93,2,32.0,0.5,0.5,0.0,0.0,0.3,33.3,0.0
2,1,2019-02-01 00:09:03,2019-02-01 00:09:16,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0
3,1,2019-02-01 00:45:38,2019-02-01 00:51:10,1,0.8,1,N,95,95,2,5.5,0.5,0.5,0.0,0.0,0.3,6.8,0.0
4,1,2019-02-01 00:25:30,2019-02-01 00:28:14,1,0.8,1,N,140,263,2,5.0,0.5,0.5,0.0,0.0,0.3,6.3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7019370,2,2019-02-28 23:29:08,2019-02-28 23:29:11,1,0.0,1,N,193,193,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7019371,2,2019-02-28 22:48:47,2019-02-28 23:50:19,1,0.0,1,N,141,193,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5
7019372,2,2019-02-28 23:41:23,2019-02-28 23:42:23,1,0.0,1,N,264,264,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7019373,2,2019-02-28 23:12:52,2019-02-28 23:14:16,1,0.0,1,N,264,193,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# add new attributes

In [4]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'],
                             format='%Y/%m/%d %H:%M', errors='coerce')
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'],
                             format='%Y/%m/%d %H:%M', errors='coerce')
df['duration'] = (df['tpep_dropoff_datetime'] -
                  df['tpep_pickup_datetime']).dt.seconds.astype(int) /60
df["revenue"] = (df['tip_amount'] + df['fare_amount'])/df['duration'] *60
df['start_hour'] =  df['tpep_pickup_datetime'].dt.hour
df['day_of_week'] = df['tpep_pickup_datetime'].dt.day_name()
df['date_hour'] = df['tpep_pickup_datetime'].dt.strftime('%d %H')

In [5]:
weather = pd.read_csv('data/2019_2_weather.csv')
weather['date_time'] = weather['date'].map(str) + " " + weather['time']
weather['date_hour'] = pd.to_datetime(weather['date_time'], format='%d %I:%M %p',
                                      errors='coerce').dt.strftime('%d %H')

weather.set_index('date_hour', inplace=True)
weather

,date,time,condition,date_time
date_hour,,,,
01 23,1,11:51 PM,Fair,1 11:51 PM
01 00,1,12:51 AM,Fair,1 12:51 AM
01 01,1,1:51 AM,Fair,1 1:51 AM
01 02,1,2:51 AM,Fair,1 2:51 AM
01 03,1,3:51 AM,Fair,1 3:51 AM
...,...,...,...,...
28 18,28,6:51 PM,Mostly Cloudy,28 6:51 PM
28 19,28,7:51 PM,Mostly Cloudy,28 7:51 PM
28 20,28,8:51 PM,Mostly Cloudy,28 8:51 PM


In [6]:
preci = ['Heavy Rain','Light Drizzle','Light Freezing Drizzle','Light Rain','Light Snow','Light Snow / Windy',
         'Rain','Snow','Snow and Sleet','Wintry Mix']

remain = ['Cloudy','Fair','Fair / Windy', 'Fog','Mostly Cloudy','Mostly Cloudy / Windy','Partly Cloudy',
          'Partly Cloudy / Windy', 'Sleet']

def precipitation_or_not(condition):
    if condition in preci:
        return 'preci'
    elif condition in remain:
        return 'remain'

weather['condition'] = weather.condition.map(precipitation_or_not)
weather

,date,time,condition,date_time
date_hour,,,,
01 23,1,11:51 PM,remain,1 11:51 PM
01 00,1,12:51 AM,remain,1 12:51 AM
01 01,1,1:51 AM,remain,1 1:51 AM
01 02,1,2:51 AM,remain,1 2:51 AM
01 03,1,3:51 AM,remain,1 3:51 AM
...,...,...,...,...
28 18,28,6:51 PM,remain,28 6:51 PM
28 19,28,7:51 PM,remain,28 7:51 PM
28 20,28,8:51 PM,remain,28 8:51 PM


In [7]:
def fill_weather(time):
    try:
        return weather['condition'].loc[time]
    except:
        return np.NaN

In [8]:
df['weather'] = df['date_hour'].map(fill_weather)


In [9]:
mta_tax = df['mta_tax'].value_counts()
mta_tax

 0.50     6971733
 0.00       38469
-0.50        9165
 3.00           4
 0.89           1
 2.80           1
 3.30           1
 24.39          1
Name: mta_tax, dtype: int64

In [10]:
improvement_surcharge = df['improvement_surcharge'].value_counts()
improvement_surcharge

 0.3    7007247
-0.3       9468
 0.0       2648
 1.0         12
Name: improvement_surcharge, dtype: int64

In [11]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration,revenue,start_hour,day_of_week,date_hour,weather
0,1,2019-02-01 00:59:04,2019-02-01 01:07:27,1,2.1,1,N,48,234,1,...,0.0,0.3,12.3,0.0,8.383333,78.727634,0,Friday,01 00,remain
1,1,2019-02-01 00:33:09,2019-02-01 01:03:58,1,9.8,1,N,230,93,2,...,0.0,0.3,33.3,0.0,30.816667,62.303948,0,Friday,01 00,remain
2,1,2019-02-01 00:09:03,2019-02-01 00:09:16,1,0.0,1,N,145,145,2,...,0.0,0.3,3.8,0.0,0.216667,692.307692,0,Friday,01 00,remain
3,1,2019-02-01 00:45:38,2019-02-01 00:51:10,1,0.8,1,N,95,95,2,...,0.0,0.3,6.8,0.0,5.533333,59.638554,0,Friday,01 00,remain
4,1,2019-02-01 00:25:30,2019-02-01 00:28:14,1,0.8,1,N,140,263,2,...,0.0,0.3,6.3,0.0,2.733333,109.756098,0,Friday,01 00,remain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7019370,2,2019-02-28 23:29:08,2019-02-28 23:29:11,1,0.0,1,N,193,193,1,...,0.0,0.0,0.0,0.0,0.050000,0.000000,23,Thursday,28 23,remain
7019371,2,2019-02-28 22:48:47,2019-02-28 23:50:19,1,0.0,1,N,141,193,2,...,0.0,0.0,0.0,2.5,61.533333,0.000000,22,Thursday,28 22,remain
7019372,2,2019-02-28 23:41:23,2019-02-28 23:42:23,1,0.0,1,N,264,264,1,...,0.0,0.0,0.0,0.0,1.000000,0.000000,23,Thursday,28 23,remain
7019373,2,2019-02-28 23:12:52,2019-02-28 23:14:16,1,0.0,1,N,264,193,1,...,0.0,0.0,0.0,0.0,1.400000,0.000000,23,Thursday,28 23,remain


# Data cleaning

df = df.loc[(df['passenger_count'] > 0) & (df['passenger_count'] <= 4) &
            (df["fare_amount"] > 0) & (df['fare_amount'] <=80) &             
            (df["duration"] >= 0.25) & (df["duration"] <= 100) &
            (df["tip_amount"] >= 0) & (df["tip_amount"] <=20) &
            (df["total_amount"]>0) & (df["total_amount"]<=100) &
            (df["trip_distance"] >= 0.01) &(df['trip_distance'] <= 30) &
            (df["revenue"] >0) &  (df["revenue"] <= 200) &           
            (df["extra"] >= 0) & (df["mta_tax"] == 0.5) & (df["congestion_surcharge"] >=0)&
            (df["tolls_amount"] >=0) &  (df["improvement_surcharge"] == 0.3) &
            (df['payment_type'] == 1) & (df['RatecodeID'] == 1)]




In [12]:
df = df.loc[(df['passenger_count'] > 0) & (df['passenger_count'] <= 4) &
            (df["fare_amount"] > 0) & (df['fare_amount'] <=80) &             
            (df["duration"] >= 0.25) & (df["duration"] <= 100) &
            (df["tip_amount"] >= 0) & (df["tip_amount"] <=20) &
            (df["total_amount"]>0) & (df["total_amount"]<=100) &
            (df["trip_distance"] >= 0.01) &(df['trip_distance'] <= 30) &
            (df["revenue"] >0) &  (df["revenue"] <= 200) &           
            (df["extra"] >= 0) & (df["mta_tax"] == 0.5) & (df["congestion_surcharge"] >=0)&
            (df["tolls_amount"] >=0) &  (df["improvement_surcharge"] == 0.3) &
            (df['payment_type'] == 1) & (df['RatecodeID'] == 1)]




In [13]:
drop_col = ['VendorID','tpep_pickup_datetime', 'tpep_dropoff_datetime','store_and_fwd_flag',
            'date_hour', 'congestion_surcharge', 'extra','mta_tax','improvement_surcharge',
            'payment_type','tolls_amount','total_amount','RatecodeID', 'PULocationID',
            'DOLocationID']
df.drop(drop_col, axis=1, inplace = True)
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)
df



C:\Users\enzon\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-13-9140f16c5c92>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


,passenger_count,trip_distance,fare_amount,tip_amount,duration,revenue,start_hour,day_of_week,weather
0,1,2.10,9.0,2.00,8.383333,78.727634,0,Friday,remain
1,2,1.80,9.0,2.05,9.800000,67.653061,0,Friday,remain
2,1,0.80,5.5,2.00,4.950000,90.909091,0,Friday,remain
3,1,9.00,26.0,7.50,16.816667,119.524281,0,Friday,remain
4,1,1.80,8.5,1.95,8.650000,72.485549,0,Friday,remain
...,...,...,...,...,...,...,...,...,...
4535200,1,7.90,24.0,10.05,20.000000,102.150000,23,Thursday,remain
4535201,1,2.86,13.0,4.20,15.950000,64.702194,23,Thursday,remain
4535202,3,3.20,14.5,3.66,18.333333,59.432727,23,Thursday,remain
4535203,1,1.22,6.5,1.00,6.183333,72.776280,23,Thursday,remain


In [14]:
df.to_feather('data/clean.feather')